# Astro 528: High-performance Computing for Astrophysics
## Lab 8, Exercise 2
## GPU Programming:  Writing Custom Kernels

### Why a Jupyter notebook?
This exercise is again a Jupyter notebook, rather than Pluto.  Why?  When using the GPU, it is possible to do things that require restarting Julia in order to recover from.  That requires shutting down the Pluto server.  The way that our Pluto servier is run from JupyterLab, that would require killing your whole JupyterLab session and requesting a new one.  This exercise has some custom CUDA kernels.  When I tested it, the notebook worked correctly.  But I encourage students to try tweaking things, and that could easily lead to some errors that require a restart.  So I decided to put this into a Jupyter notebook.  In Jupyter, you can go to the *Kernel* menu and select *Restart Kernel* to get a new kernel and try again.

### Setting up the GPU
Remember that when using Jupyter, the order in which you execute cells affects the state of the notebook.  (In contrast to in Pluto, where it figures out the order that cells are to be run, so we can put implementation details at the bottom of the notebook, where they're less distracting.)
Therefore, we place code in the order that it should be run.  We'll start loading the packages we'll use for writing our custom GPU kernels.

In [23]:
import Pkg;  Pkg.offline() # Tell Julia not to try to find things on the internet
using KernelAbstractions   # Allows writing code once for different brand GPUs
using CUDA, CUDAKernels    # for NVIDIA GPUs
using BenchmarkTools       # for timing
using Markdown             # for responces

As before we will check that the system we're running on has an NVIDIA GPU that we can use and check what's installed.

In [24]:
CUDA.devices()

CUDA.DeviceIterator() for 1 devices:
0. Tesla P100-PCIE-12GB

In [25]:
CUDA.versioninfo()

CUDA toolkit 11.7, artifact installation
NVIDIA driver 535.104.12, for CUDA 12.2
CUDA driver 12.2

Libraries: 
- CUBLAS: 11.10.1
- CURAND: 10.2.10
- CUFFT: 10.7.2
- CUSOLVER: 11.3.5
- CUSPARSE: 11.7.3
- CUPTI: 17.0.0
- NVML: 12.0.0+535.104.12
- CUDNN: 8.30.2 (for CUDA 11.5.0)
- CUTENSOR: 1.4.0 (for CUDA 11.5.0)

Toolchain:
- Julia: 1.9.2
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: Tesla P100-PCIE-12GB (sm_60, 10.566 GiB / 12.000 GiB available)


We'll make use of the GPU's warpsize later in the exericse, so let's get it now.

In [26]:
warpsize(CUDA.device())

32

## CPU version of code to run in parallel 
We'll write two functions that we'll convert into GPU kernels.  The first, `calc_rv_circ`, will be a simple function, so we keep things simple.  The second, `calc_rv_kepler`, will call other several user-written functions.  We'll even place the code to solve Kepler's equation in a module to help with code maintainability.

In [27]:
function calc_rv_circ(t::Real; param::NamedTuple )
    P = param.P
    K = param.K
    M0 = param.M0
    mean_anom = t*2π/P-M0
    rv = K*sin(mean_anom)
end

calc_rv_circ (generic function with 1 method)

### Code for computing radial velocity of a Keplerian orbit

In [28]:
"Solve Computing Eccentric Anomaly from Mean Anomally via Kepler's Equation"
module KeplerEqn
using KernelAbstractions  # since we'll use KernelAbstractions to get slightly better performance.
export calc_ecc_anom

"""   ecc_anom_init_guess_danby(M, ecc)
Initial guess for eccentric anomaly given mean anomaly (M) and eccentricity (ecc)
    Based on "The Solution of Kepler's Equations - Part Three"  
    Danby, J. M. A. (1987) Journal: Celestial Mechanics, Volume 40, Issue 3-4, pp. 303-312  1987C
eMec..40..303D
"""
function ecc_anom_init_guess_danby(M, ecc)
    # Match type of ecc to avoid type conversions on GPU.
    k = convert(typeof(ecc),0.85)
    # Julia's pi is irrational, so we need to convert to a type the GPU can use.
    local my_pi = convert(typeof(M),pi)
    if(M<zero(M)) M += 2*my_pi end
    (M<my_pi) ? M + k*ecc : M - k*ecc;
end

"""   update_ecc_anom_laguerre(E, M, ecc)
Update the current guess (E) for the solution to Kepler's equation given mean anomaly (M) and ecc
entricity (ecc)
   Based on "An Improved Algorithm due to Laguerre for the Solution of Kepler's Equation"
   Conway, B. A.  (1986) Celestial Mechanics, Volume 39, Issue 2, pp.199-211  1986CeMec..39..199C
"""
function update_ecc_anom_laguerre(E, M, ecc)
  es, ec = ecc.*sincos(E)
  F = (E-es)-M
  Fp = one(E)-ec
  Fpp = es
  n = 5
  root = sqrt(abs((n-1)*((n-1)*Fp*Fp-n*F*Fpp)))
  denom = Fp>zero(E) ? Fp+root : Fp-root
  return E-n*F/denom
end

const default_ecc_anom_tol = 1e-8
"Loop to update the current estimate of the solution to Kepler's equation."
function calc_ecc_anom(mean_anom, ecc, tol = default_ecc_anom_tol )
    default_max_its_laguerre = 200
    max_its = default_max_its_laguerre
    @assert zero(ecc) <= ecc < one(ecc)
    @assert tol*100 <= one(tol)
    # Julia's pi is irrational, so we need to convert to a type the GPU can use.
    M = mod(mean_anom,convert(typeof(mean_anom),2*pi))
    #M = rem2pi(mean_anom,RoundNearest)
    E = ecc_anom_init_guess_danby(M,ecc)
    # Slight optimization from our original CPU code here, to tell the GPU to try to unroll the first several itterations of the for loop. 
    #for i in 1:max_its
    KernelAbstractions.Extras.@unroll 6 for i in 1:max_its
       E_old = E
       E = update_ecc_anom_laguerre(E_old, M, ecc)
       if abs(E-E_old)<convert(typeof(mean_anom),tol) break end
    end
    return E
end

end # module KeplerEqn

Main.KeplerEqn

In [29]:
function calc_true_anom(ecc_anom::Real, e::Real)
    true_anom = 2*atan(sqrt((1+e)/(1-e))*tan(ecc_anom/2))
end

calc_true_anom (generic function with 1 method)

In [30]:
function calc_rv_kepler(t::Real; param::NamedTuple )
    P = param.P
    K = param.K
    ecc = param.e
    ω = param.ω
    M0 = param.M0
    mean_anom = t*2π/P-M0
    ecc_anom = KeplerEqn.calc_ecc_anom(mean_anom,ecc)
    true_anom = calc_true_anom(ecc_anom,ecc)
    rv = (cos(ω+true_anom)+ecc*cos(ω)) * K/sqrt((1-ecc)*(1+ecc))
end

calc_rv_kepler (generic function with 1 method)

## Writing GPU kernels with KernelAbstractions.jl
The CPU version of `calc_rv_circ_kernel` and `calc_rv_kepler_kernel` operate on scalars.  For GPU calculations to be efficient, we need them to operate on arrays.  For some simple use cases (like the examples below), we could just use `map`.  However, it is useful to start simple when demonstrating how to write a custom GPU kernel.  Below we'll using the `KernelAbstractions` `@kernel` macro to simplify writing our GPU kernel.  For example, rather than calculating the index to operate on explicitly from the thread and block indices, we'll use the `@index` macro to get the index for each thread to operate on.  There's also a useful `@Const` macro that allows us to specify that the data in our input arrays will remain constant, allowing for memory-related optimizations that aren't always possible. It will be up to us when we call the kernel to make sure that there as many threads as elements of `y` and `times`.  

In [31]:
KernelAbstractions.@kernel function calc_rv_circ_kernel(y, @Const(times), @Const(param) )
    I = @index(Global)
    t = times[I]
    y[I] = calc_rv_circ(t, param=param)
end

In [32]:
KernelAbstractions.@kernel function calc_rv_kepler_kernel(y, @Const(times), @Const(param) )
    I = @index(Global)
    t = times[I]
    y[I] = calc_rv_kepler(t,param=param)
end

One great feature of the `KernelAbstractions.jl` package is that we can the core code for kernels once, and turn them into kernels that can run on a CPU, an NVIDIA GPU or an AMD GPU. 
In order to get a kernel that can be executed, we call the function returned by the `@kernel` macro, specifying the hardware we will use to execute the kernel.  
Optionally, we can specify the *workgroup size* and the size of the global indices to be used, when we request the kernel.  If we don't provide that info now, then we'll need to provide it at runtime. As we discussed previously, its often useful for the compiler to have information at compile time.  Therefore, we'll specify the workgroup size at compile time in building our kernels below.  We'll leave the size of the global indices as a dynamic parameter, so that we don't need to recompile a kernel for each problem size.

In [33]:
cpu_kernel_calc_rv_circ! = calc_rv_circ_kernel(CPU(), 16)

KernelAbstractions.Kernel{CPU, KernelAbstractions.NDIteration.StaticSize{(16,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(cpu_calc_rv_circ_kernel)}(cpu_calc_rv_circ_kernel)

In [34]:
cpu_kernel_calc_rv_kepler! = calc_rv_kepler_kernel(CPU(), 16)

KernelAbstractions.Kernel{CPU, KernelAbstractions.NDIteration.StaticSize{(16,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(cpu_calc_rv_kepler_kernel)}(cpu_calc_rv_kepler_kernel)

## Generate data for testing kernels
Let's generate some simulated data for testing our CPU kernels.  Below are functions that will make it easy to generate datasets of various sizes later in the exercise.

In [35]:
function generate_obs_data(times::AbstractArray; param, σ_obs, model)
    num_obs = length(times)
    rv_true = model.(times, param = param)
    rv_obs = rv_true .+ σ_obs.*randn(num_obs)
    obs_data = (; t=times, rv=rv_obs, σ=σ_obs.*ones(num_obs) )
end

function generate_obs_data(;time_span, num_obs, param, σ_obs, model)
    days_in_year = 365.2425
    times = sort(time_span*days_in_year*rand(num_obs))
    generate_obs_data(times, param=param, σ_obs=σ_obs, model=model)
end

generate_obs_data (generic function with 2 methods)

In [36]:
begin
    P_true = 3.0
    K_true = 5.0
    e_true = 0.4
    ω_true = π/4
    M0_true = π/4
    θ_true  = (;P=P_true, K=K_true, e=e_true, ω=ω_true, M0=M0_true )
end

(P = 3.0, K = 5.0, e = 0.4, ω = 0.7853981633974483, M0 = 0.7853981633974483)

In [37]:
n_obs = 10_000
time_span_in_years = 1
obs_data = generate_obs_data(time_span=time_span_in_years, num_obs=n_obs, param=θ_true, σ_obs=1.0, model=calc_rv_kepler);

It's often nice to visualize our data, just to make sure our code is doing what we expect.  I've commented out the plotting code, so the pbs script will work.  Also, note that if we plotted all the data, it would take a *very* long time, you'll be better off just ploting a random sample of the points.

In [38]:
#using Plots

In [39]:
#=
idx_plt = length(obs_data.t) <= 100 ? (1:length(obs_data.t))  : rand(1:length(obs_data.t),100) 
plt = scatter(obs_data.t[idx_plt],obs_data.rv[idx_plt],yerr=obs_data.σ[idx_plt],legend=:none,ms=2)
xlabel!(plt,"Time (d)")
ylabel!(plt,"RV (m/s)")
title!(plt,"Simulated Data")
=#

## Executing a kernel
First, we'll make sure that the CPU kernel we generated with KernelAbstractions gives similar results.  Since our kernel needs an array to write it's output to, we'll allocate memory for that.  Then we'll pass the optional arguement `ndrange` to tell it the size of the global indices that it should use for the times and outputs.  Kernel calls can be asynchronous, so they return an *event* that can be used to check whether the kernel has completed.  We call `wait` on the event to make sure our kernel has completed its work before using the results.

In [40]:
output_cpu = similar(obs_data.t)
cpu_event = cpu_kernel_calc_rv_circ!(output_cpu, obs_data.t, θ_true, ndrange=size(obs_data.t))
wait(cpu_event)

Now we'll try performing the same calculation on the GPU.  We generate a GPU kernel for NVIDIA GPUs by passing `CUDADevice()` instead of `CPU()`.  We'll specify a workgroup size equal to the warpsize on our GPUs.  

In [41]:
output_gpu_d = CUDA.zeros(length(obs_data.t));
obs_times_d = cu(obs_data.t);

In [42]:
gpu_kernel_calc_rv_circ! = calc_rv_circ_kernel(CUDADevice(), 32)

KernelAbstractions.Kernel{CUDADevice{false, false}, KernelAbstractions.NDIteration.StaticSize{(32,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(gpu_calc_rv_circ_kernel)}(gpu_calc_rv_circ_kernel)

In [43]:
gpu_event = gpu_kernel_calc_rv_circ!(output_gpu_d, obs_times_d, θ_true, ndrange=size(obs_times_d))
wait(gpu_event)

## Accuracy of GPU calculations
Before we start benchmarking, let's check the the results from the GPU kernel are accurate.    

In [44]:
maximum(abs.(collect(output_gpu_d).-output_cpu))

0.00015841244088615758

That's likely larger than we'd normally expect.  What could have caused it.  It's good to check the types of the arrays on the device.

In [45]:
typeof(obs_times_d), typeof(output_gpu_d)

(CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}, CuArray{Float32, 1, CUDA.Mem.DeviceBuffer})

Why?  Look back at where we allocated `obs_times_d` and `output_gpu_d`.  `cu` defaults to sending arrays as Float32's rather than Float64's.  We can be explicit about what type we want.  Additionally, the default element type for `CUDA.zeros` is Float32, rather than Float64 (like it is for `Base.zeros`).  

In [46]:
obs_times_d = convert(CuArray{Float64,1},obs_data.t);
output_gpu_d64 = CUDA.zeros(Float64,length(obs_data.t));
gpu_event = gpu_kernel_calc_rv_circ!(output_gpu_d64, obs_times_d, θ_true, ndrange=size(obs_times_d))
wait(gpu_event)
maximum(abs.(collect(output_gpu_d64).-output_cpu))

8.881784197001252e-16

That should be much better.  (It's still not zero, since the GPU defaults to allowing the optimizer to perform optimizations that are not IEEE-compliant.) 

## Comparing CPU & GPU Performance 
Next, we'll comapre the time to execute the CPU and GPU kernels.  In the test below, we'll compute the predicted radial velcoity for the first `num_obs_to_eval` time.  The default code below is for the full set of observation times.  But you may wish to try comparing the performance for fewer observations (without having to regenerate the simulated data).

In [47]:
num_obs_to_eval = length(obs_data.t)

10000

In [48]:
@benchmark  wait(cpu_kernel_calc_rv_circ!($output_cpu, $obs_data.t, $θ_true, ndrange=$num_obs_to_eval )) seconds=1

BenchmarkTools.Trial: 9520 samples with 1 evaluation.
 Range (min … max):   51.553 μs …   5.493 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     103.124 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   101.654 μs ± 108.991 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▃▃▄▂                 █▅▆▇▅▄▁                                 ▂
  ██████▆▆▅▅▄▄▁▄▁▄▃▁▃▁▇█████████▇▇██▇▆▆▄▃▁▄▃▁▃▁▁▃▁▃▁▁▅▆▅█▇▇█▆▆▅ █
  51.6 μs       Histogram: log(frequency) by time        188 μs <

 Memory estimate: 4.11 KiB, allocs estimate: 60.

In [49]:
@benchmark wait(gpu_kernel_calc_rv_circ!($output_gpu_d64, $obs_times_d, $θ_true, ndrange=$num_obs_to_eval)) seconds=1

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  20.362 μs …  1.764 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.265 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.012 μs ± 18.226 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▆███▆▅▄▂▁▂▃▃▃▄▃▂▂▂▂▁▁                    ▁▂▃▂▁            ▂
  ▇███████████████████████▇▆▅▄▅▅▅▆▆▅▃▄▁▃▅▃▃▅▇███████▆▆▅▇███▇▇ █
  20.4 μs      Histogram: log(frequency) by time      42.2 μs <

 Memory estimate: 2.89 KiB, allocs estimate: 60.

2a.  How much faster was performing the computations on the GPU than on the CPU? 

In [50]:
response_2a = missing # md"Insert response"

missing

## Performance as a function of workgroup size
When we generate a kernel to run on the GPU, the workgroup size can make a significant difference in the performance.  My understanding is that the workgroup size is equivalent to the block size on NVIDIA GPUs.  response_2a = missing # md"Insert response"

2b.  What do you predict for the compute time for the same GPU kernel, except with the workgroup size set to 1?

In [51]:
response_2b = missing # md"Insert response"

missing

In [52]:
gpu_kernel_calc_rv_circ_alt! = calc_rv_circ_kernel(CUDADevice(), 1)

KernelAbstractions.Kernel{CUDADevice{false, false}, KernelAbstractions.NDIteration.StaticSize{(1,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(gpu_calc_rv_circ_kernel)}(gpu_calc_rv_circ_kernel)

In [53]:
@benchmark wait(gpu_kernel_calc_rv_circ_alt!($output_gpu_d, $obs_times_d, $θ_true, ndrange=$num_obs_to_eval)) seconds=1

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  44.399 μs … 168.995 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.449 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   49.093 μs ±   4.876 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▅▇█▆▂                                                     
  ▁▂▅██████▇▅▅▅▆▆▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁ ▂
  44.4 μs         Histogram: frequency by time         67.5 μs <

 Memory estimate: 2.91 KiB, allocs estimate: 61.

2c.  How did the performance with a workgrop size of 1 compare to your predictions?  

In [54]:
response_2c = missing # md"Insert response"

missing

If you're likely to be using a GPU for parallelizing your project code, then try adjusting the workgroup size to other values (between 1 and twice the warpsize) and see how it affects the performance.

## Comparing performance of kernels
So far, we've been benchmarking a relatively simple kernel (some arithmetic and one trig function).  Now, we'll try switching to computing the radial velocity assuming a Keplerian orbit.  First, we'll use a CPU version of the kernel.  

In [55]:
@benchmark wait(cpu_kernel_calc_rv_kepler!($output_cpu, $obs_data.t, $θ_true, ndrange=size($obs_data.t))) seconds=1

BenchmarkTools.Trial: 795 samples with 1 evaluation.
 Range (min … max):  1.181 ms …  2.764 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.231 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.249 ms ± 99.549 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▄█ ▁                                                   
  ▄▃▃▃▃████▇▄▄▃▃▂▂▂▂▁▂▂▁▁▁▂▁▁▂▂▁▁▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▂▁▁▂▂▂▁▁▂▂▂▂ ▃
  1.18 ms        Histogram: frequency by time        1.61 ms <

 Memory estimate: 4.09 KiB, allocs estimate: 59.

Next, we'll create a GPU kernel to benchmark the Keplerian calculation on the GPU.

In [56]:
gpu_kernel_calc_rv_kepler! = calc_rv_kepler_kernel(CUDAKernels.CUDADevice(), 32)

KernelAbstractions.Kernel{CUDADevice{false, false}, KernelAbstractions.NDIteration.StaticSize{(32,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(gpu_calc_rv_kepler_kernel)}(gpu_calc_rv_kepler_kernel)

2d.   Before you run the benchmarks, what do you expect for the GPU performance if we use a workgroup size equal to the warpsize?  What if we use a workgroup size of 1?  Explain your reasoning.  

In [57]:
response_2d = missing # md"Insert response"

missing

In [58]:
@benchmark wait(gpu_kernel_calc_rv_kepler!($output_gpu_d, $obs_times_d, $θ_true, ndrange=size($obs_times_d))) seconds=1

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  25.679 μs …  1.499 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.596 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   30.881 μs ± 15.688 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▃█▇▃                                                 
  ▁▁▂▃▆▇▇▇████▅▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁ ▂
  25.7 μs         Histogram: frequency by time        49.1 μs <

 Memory estimate: 2.89 KiB, allocs estimate: 60.

In [59]:
gpu_kernel_calc_rv_kepler_alt! = calc_rv_kepler_kernel(CUDAKernels.CUDADevice(), 1)
@benchmark wait(gpu_kernel_calc_rv_kepler_alt!($output_gpu_d, $obs_times_d, $θ_true, ndrange=size($obs_times_d))) seconds=1

BenchmarkTools.Trial: 7741 samples with 1 evaluation.
 Range (min … max):  101.200 μs …  15.623 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     104.850 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   126.114 μs ± 290.593 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▆▄▂▁▃▃▁▁                                                    ▁
  ██████████▇▆▅▅▄▄▁▃▁▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▄▁▄▃▃▁▁▁▁▁▁▁▃▁▁▃▅▆▆▅▆▆ █
  101 μs        Histogram: log(frequency) by time        263 μs <

 Memory estimate: 2.91 KiB, allocs estimate: 61.

2e.  How did the benchmarking results for the GPU version compare to your expectations?  What could explain the differences? 

In [60]:
response_2e = missing # md"Insert response"

missing

## High-level GPU Programming with FLoops

In [Lab 6](https://github.com/PsuAstro528/lab6-start), we saw how we could write parallel for either serial, multithreaded or distributed architectures using [FLoops.jl](https://juliafolds.github.io/FLoops.jl/dev/).   The [FoldsCUDA.jl](https://github.com/JuliaFolds/FoldsCUDA.jl) package provided an executor that allows FLoops to compile code for the GPU.  THe usual limitations about GPU kernels not being able to allocate memory still apply.  In order for the GPU kernel to be allowed to write to GPU arrays, we use the `referencable()` function and an unusual syntax shown below.  

In [61]:
using FLoops
using CUDA, FoldsCUDA
using Referenceables: referenceable

In [62]:
function calc_rv_kepler_floops(output::AbstractArray, times::AbstractArray, param; ex =  ThreadedEx() )
     @floop ex for (t_i,rv_i) in zip(referenceable(times),referenceable(output))
        rv_i[] = calc_rv_kepler(t_i[],param=param)
    end
    output
end

calc_rv_kepler_floops (generic function with 1 method)

In [63]:
output_cpu_floops = similar(obs_data.t)
calc_rv_kepler_floops(output_cpu_floops,obs_data.t,θ_true)
all(output_cpu_floops .≈ output_cpu)

true

In [64]:
output_gpu_floops = CUDA.similar(obs_times_d)
calc_rv_kepler_floops(output_gpu_floops,obs_times_d,θ_true, ex=CUDAEx())
all(collect(output_gpu_floops) .≈ output_cpu)

true

## Improving performance by performing reductions on GPU
In the previous calculations, there was a substantial ammount of data to be transfered back from the GPU to CPU.  Often, we don't need all the data to be moved back to the CPU, since we're primarily interested in one or more summary statistics.  For example, we might be interested in the chi-squared statistic for comparing our model predictions to the data.  In that case, we'll only need to return a very small ammount of data from the GPU back to the CPU.  Below, we'll see how that affects the performance.  

First, let's try calculating chi-squared the obvious way on the CPU.

In [65]:
function calc_chisq_rv_circ_cpu_simple(data, param)
    @assert length(data.t) == length(data.rv) == length(data.σ) 
    sum(((calc_rv_circ.(data.t, param=param).-data.rv)./data.σ).^2)
end

calc_chisq_rv_circ_cpu_simple (generic function with 1 method)

In [66]:
function calc_chisq_rv_kepler_cpu_simple(data, param)
    @assert length(data.t) == length(data.rv) == length(data.σ) 
    sum(((calc_rv_kepler.(data.t, param=param).-data.rv)./data.σ).^2)
end

calc_chisq_rv_kepler_cpu_simple (generic function with 1 method)

In [67]:
chisq_from_cpu_simple = calc_chisq_rv_kepler_cpu_simple(obs_data, θ_true)

9864.494571352594

In [68]:
@benchmark calc_chisq_rv_kepler_cpu_simple($obs_data, $θ_true) seconds=1

BenchmarkTools.Trial: 461 samples with 1 evaluation.
 Range (min … max):  2.125 ms …  2.307 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.159 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.166 ms ± 42.193 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▅▂▁      ▇▇▄▁   ▁            ▂         ▁                  
  █████▄▅▅▅▆▅████▇▆▅█▅▆▁██▇█▅▅▅▆▆█▇▆▇▅▆▄▅▅▄█▄▅▁▆▅▅▅█▅▁▄▆▁▆▄▄ ▇
  2.13 ms      Histogram: log(frequency) by time     2.29 ms <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [69]:
function cpu_calc_chisq_kepler(t::AbstractArray, rv_obs::AbstractArray, σ::AbstractArray, θ; workspace = missing )
    @assert length(t) == length(rv_obs) == length(σ)
    @assert ismissing(workspace) || length(t) == length(workspace)
    rv_pred = ismissing(workspace) ? similar(rv_obs) : workspace
    wait(cpu_kernel_calc_rv_kepler!(rv_pred, t, θ, ndrange=size(t)))
    χ² = sum(((rv_pred.-rv_obs)./σ).^2)
end

function cpu_calc_chisq_kepler(data::NamedTuple, θ; workspace = missing)
    cpu_calc_chisq_kepler(data.t, data.rv, data.σ, θ, workspace = workspace)
end

cpu_calc_chisq_kepler (generic function with 2 methods)

In [70]:
cpu_calc_chisq_kepler(obs_data, θ_true) ≈ calc_chisq_rv_kepler_cpu_simple(obs_data, θ_true)

true

Now let's try the same calculation on the CPU using the CPU kernel built by `KernelAbstractions`.

In [71]:
@benchmark cpu_calc_chisq_kepler($obs_data,$θ_true) seconds=1

BenchmarkTools.Trial: 780 samples with 1 evaluation.
 Range (min … max):  1.210 ms …   4.418 ms  ┊ GC (min … max): 0.00% … 70.10%
 Time  (median):     1.256 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.274 ms ± 164.078 μs  ┊ GC (mean ± σ):  0.61% ±  3.53%

             ▃█                                                
  ▅▄▃▂▃▃▃▃▃▃▄██▆▅▅▅▇▅▄▂▃▄▄▃▃▃▃▂▃▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▂ ▃
  1.21 ms         Histogram: frequency by time        1.42 ms <

 Memory estimate: 160.47 KiB, allocs estimate: 64.

That was likely much faster than the simple way!  How is that possible?  The `KernelAbstractions` package is also parallelizing the calculation over multiple threads (assuming that you launched julia using multipel threads).   Let's check how many threads are avaliable.

In [72]:
Threads.nthreads()

4

Since KernelAbstractions is primarily geared towards GPU computing, it might not generate code that is as efficient as some other packages designed for multi-threaded computing.  Nevertheless, it likely does pretty well on a relatively simple example like this.

### Performing reduction using Array interface to GPU
Since our GPU kernel writes its output to an array on the GPU, we can use the array interface to the GPU to perform the reduction.

In [73]:
function gpu_calc_chisq_kepler(t::CuArray, rv_obs::CuArray, σ::CuArray, θ; workspace::Union{Missing,CuArray} = similar(rv_obs) )
    @assert length(t) == length(rv_obs) == length(σ)
    @assert ismissing(workspace) || length(t) == length(workspace)
    rv_pred = ismissing(workspace) ? similar(rv_obs) : workspace
    gpu_kernel_calc_rv_kepler!(rv_pred, t, θ, ndrange=size(t))
    χ² = sum(((rv_pred.-rv_obs)./σ).^2)
end


gpu_calc_chisq_kepler (generic function with 1 method)

 Note that, previously, we didn't transfer the observed velocity and uncertainty to the GPU.  In order to perform the reduction on the GPU, we'll want to do that.  To make it more convenient, we can write a wrapper function that accepts AbstractArrays and transfers them to the GPU if necessary.  We'll also provide an optional workspace parameter, so that we can pass a preallocated array to store the predicted radial velocities in.

In [74]:
function gpu_calc_chisq_kepler(t::AbstractArray, rv_obs::AbstractArray, σ::AbstractArray, θ; workspace = missing )
    t_d = isa(t,CuArray) ? t : convert(CuArray{eltype(t),1},t)
    rv_obs_d = isa(rv_obs,CuArray) ? rv_obs : convert(CuArray{eltype(rv_obs),1},rv_obs)
    σ_d = isa(σ,CuArray) ? σ : convert(CuArray{eltype(σ),1},σ)
    workspace_d = isa(workspace, CuArray) ? workspace : similar(t_d)
    gpu_calc_chisq_kepler(t_d,rv_obs_d,σ_d,θ, workspace=workspace_d)
end


gpu_calc_chisq_kepler (generic function with 2 methods)

Thinking back to our best-practices for scientific software development readings and discussion, passing several arrays to a function can be a bit dangerous, since we have to remember the correct order.  GPU kernels have some limitations, but we can compensate by wrapping our GPU calls with CPU-level functions that have a safer interface.  For example, we can make a nice wrapper function that takes a NamedTuple of arrays and a set of parameters, with an optional named parameter for a pre-allocated workspace.

In [75]:
function gpu_calc_chisq_kepler(data::NamedTuple, θ; workspace = missing)
    gpu_calc_chisq_kepler(data.t, data.rv, data.σ, θ, workspace = workspace)
end

gpu_calc_chisq_kepler (generic function with 3 methods)

Before benchmarking, let's make sure that the results are acceptably accurate.

In [76]:
gpu_calc_chisq_kepler(obs_data, θ_true) ≈ cpu_calc_chisq_kepler(obs_data, θ_true)

true

2f.  What do you predict for the time required to transfer the input data to the GPU, to compute the predicted velocities (allocating GPU memory to hold the results), to calcualte chi-squared on the GPU and to return just the chi-squared value to the CPU?

In [77]:
response_2f = missing # md"Insert response"

missing

In [78]:
@benchmark gpu_calc_chisq_kepler($obs_data, $θ_true) seconds=1

BenchmarkTools.Trial: 5808 samples with 1 evaluation.
 Range (min … max):  140.314 μs …  54.483 ms  ┊ GC (min … max): 0.00% … 25.73%
 Time  (median):     152.884 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   172.316 μs ± 983.367 μs  ┊ GC (mean ± σ):  2.83% ±  0.50%

          ▁▃▄▄▃▃▄▄▃▅█▇█▆▆▇▆▄▃▁                                   
  ▁▁▂▃▃▅▇█████████████████████▇█▆▇▅▆▄▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁ ▄
  140 μs           Histogram: frequency by time          178 μs <

 Memory estimate: 14.02 KiB, allocs estimate: 282.

2g.  How did the benchmarking results compare to your prediction?  What might explain any differences?

In [79]:
response_2g = missing # md"Insert response"

missing

Now, we'll repeat the benchmarking, but using input data already loaded onto the GPU and a pre-alocated workspace on the GPU.  The function below helps with that.  Don't worry about the non-obvious syntax.

In [80]:
function convert_namedtuple_of_arrays_to_namedtuple_of_cuarrays(θ::NamedTuple{NTK,NTVT}) where { NTK, T<:Real, N1, A<:AbstractArray{T,N1}, N2, NTVT<:NTuple{N2,A} }
    (; zip(keys(θ), convert.(CuArray{T,N1},values(θ)) )... )
end

convert_namedtuple_of_arrays_to_namedtuple_of_cuarrays (generic function with 1 method)

In [81]:
obs_data_gpu = convert_namedtuple_of_arrays_to_namedtuple_of_cuarrays(obs_data)

(t = [0.07179972181638739, 0.1192269192785277, 0.2265719591891274, 0.22726862115199276, 0.29870507787108025, 0.30541114085923204, 0.32200756952517234, 0.4257491526322243, 0.4314835296511047, 0.5018662067645855  …  364.78977981588463, 364.8522408167147, 364.9454946805053, 364.9947390765156, 364.9994983999964, 365.0286940813796, 365.0966998487043, 365.1455233058268, 365.15183380185977, 365.19437829559513], rv = [7.636194948702828, 6.271668734627825, 7.414347065264702, 8.525580645712742, 5.658250325835963, 7.623441405982858, 8.512024856458035, 4.426018990319287, 4.013614360348758, 1.6088917635446198  …  -4.001993154895516, -1.9725763362321724, -1.6950837390883056, -1.5518832987633953, -0.5970177126095908, -1.869890396077734, -2.233019483949304, -2.602545739838835, -1.009721078115015, -1.795082532716695], σ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

2h.  What do you predict for the time required to compute the predicted velocities (using a pre-allocated workspace on the GPU), to compute chi-squared on the GPU and to return just the chi-squared value to the CPU?  

In [82]:
response_2h = missing # md"Insert response"

missing

In [83]:
@benchmark gpu_calc_chisq_kepler($obs_data_gpu, $θ_true, workspace = $output_gpu_d64) seconds=1

BenchmarkTools.Trial: 9609 samples with 1 evaluation.
 Range (min … max):   81.278 μs …  36.805 ms  ┊ GC (min … max): 0.00% … 33.58%
 Time  (median):      90.328 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   100.205 μs ± 525.218 μs  ┊ GC (mean ± σ):  2.58% ±  0.49%

        ▃▄▆█▆▅▃▃▁                                                
  ▁▂▂▄▆▇█████████▇█▆▆▅▆▆▆▆▆▆▆▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁ ▃
  81.3 μs          Histogram: frequency by time          116 μs <

 Memory estimate: 12.16 KiB, allocs estimate: 223.

2i.  How did the benchmarking results compare to your prediction?  What might explain any differences?

In [84]:
response_2i = missing # md"Insert response"

missing

For examples of how to fuse parallel calculations and reductions into a single GPU kernel call using FLoops, see [this example](https://juliafolds.github.io/FoldsCUDA.jl/dev/examples/inplace_mutation_with_referenceables/#Fusing-reduction-and-mutationg).

## Evaluating many models at once
In the previous examples, we got a good performance speed-up when we computed a million predicted velocities in one GPU call.  Most stars won't have nearly that many observations.  If we only had a few hundred observations and used the code above, we would not get nearly as good performance out of the CPU.  (Feel free to try repeating the above benchmarks, changing n_obs.  Remember, you'd need to rerun all the cells that are affected.)  Is there a way that GPU computing might still be useful?  

Typically, we don't just want to evalute one model, but rather need to evaluate thousand, millions or billions of models to find those that are a good match to the observations.  This creates a second opportunity for parallelization.  We can have each thread evaluate the predicted velocity for one observation time and one set of model parameters.  Let's try that.  

First, we'll generate a smaller dataset for testing, and load it onto the GPU.

In [85]:
n_obs_small = 256
obs_data_small = generate_obs_data(time_span=time_span_in_years, num_obs=n_obs_small, param=θ_true, σ_obs=1.0, model=calc_rv_kepler);
obs_data_small_gpu = convert_namedtuple_of_arrays_to_namedtuple_of_cuarrays(obs_data_small)

(t = [2.73403915051228, 4.969203819555879, 5.4688898240562756, 6.476838970709489, 6.8407747956369604, 9.076183991590536, 9.388386792405043, 9.847104577153669, 11.153718675871396, 12.538926762635613  …  351.3100212016383, 353.5062966813871, 357.57358773762996, 358.968694199566, 359.0450417154525, 361.4808008506871, 362.18053164650115, 362.9395706685908, 363.1485563886233, 364.34679897781615], rv = [4.206624109320136, -0.9777834540035577, 1.730470222703137, 2.8059865731933376, -2.0237141598313206, 6.311131321624286, 4.590005681773708, -1.8182003940526186, -1.5055958243967749, 1.05471392637463  …  6.030491143778537, 0.19322121059991548, 1.0531613588617117, -2.8688873829923107, -0.9302376384580748, -2.4065346256048756, 0.13880885504346385, 4.387424719390905, 5.825302633605026, -3.8945828014026977], σ = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

Now, let's generate a lot of different sets of model parameters to try evaluating.

In [86]:
num_models = 2048 
σ_P = P_true*1e-4
σ_K = K_true*1e-2
σ_e = 0.1
σ_ω = 2π*0.1
σ_M0 = 2π*0.1
θ_eval = (;P=clamp.(P_true.+σ_P.*randn(num_models), 0.0, Inf),
    K=clamp.(K_true.+σ_K.*randn(num_models), 0.0, Inf),
    e=clamp.(e_true.+σ_e.*randn(num_models), 0.0, 1.0),
    ω=ω_true.+σ_ω.*randn(num_models),
    M0=M0_true.+σ_M0.*randn(num_models) );

Next, we'll write a custom kernel that computes the predicted velocity for one observation time and one set of model parameters.  
We'll need to specify a problem size that is a tuple, with the first dimension being the number of times and the second dimension being the number of models to evaluate.

In [87]:
KernelAbstractions.@kernel function calc_rv_kepler_kernel_many_models(y, @Const(times), @Const(P), @Const(K), @Const(ecc), @Const(ω), @Const(M0) )
    I, J = @index(Global, NTuple)
    t_I = times[I]
    param_J = (; P=P[J], K=K[J], e=ecc[J], ω=ω[J], M0=M0[J] )
    y[I,J] = calc_rv_kepler(t_I,param=param_J)
end

calc_rv_kepler_kernel_many_models (generic function with 4 methods)

We'll create a CPU version of the kernel,  allocate the memory for it to store its results in, and test the CPU kernel.

In [88]:
cpu_kernel_calc_rv_kepler_many_models! = calc_rv_kepler_kernel_many_models(CPU(), 16)

KernelAbstractions.Kernel{CPU, KernelAbstractions.NDIteration.StaticSize{(16,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(cpu_calc_rv_kepler_kernel_many_models)}(cpu_calc_rv_kepler_kernel_many_models)

In [89]:
output_many_models_cpu = zeros(n_obs_small, num_models);

In [90]:
wait(cpu_kernel_calc_rv_kepler_many_models!(output_many_models_cpu, obs_data_small.t, θ_eval.P, θ_eval.K, θ_eval.e, θ_eval.ω, θ_eval.M0, ndrange=( n_obs_small, num_models ) ))

Now, we'll create a GPU version of the kernel, allocate the memory for it to store its results in, load the model parameters to be evaluated onto the GPU, and test Ghe CPU kernel.

In [91]:
gpu_kernel_calc_rv_kepler_many_models! = calc_rv_kepler_kernel_many_models(CUDAKernels.CUDADevice(), 32)

KernelAbstractions.Kernel{CUDADevice{false, false}, KernelAbstractions.NDIteration.StaticSize{(32,)}, KernelAbstractions.NDIteration.DynamicSize, typeof(gpu_calc_rv_kepler_kernel_many_models)}(gpu_calc_rv_kepler_kernel_many_models)

In [92]:
output_many_models_gpu = CUDA.zeros(Float64,n_obs_small, num_models);

In [93]:
θ_eval_gpu = convert_namedtuple_of_arrays_to_namedtuple_of_cuarrays(θ_eval);

In [94]:
wait(gpu_kernel_calc_rv_kepler_many_models!(output_many_models_gpu, obs_data_small_gpu.t, θ_eval_gpu.P, θ_eval_gpu.K, θ_eval_gpu.e, θ_eval_gpu.ω, θ_eval_gpu.M0, ndrange=( n_obs_small, num_models ) ))

A quick check that the results are consistent given expected limitations of floating point arithmetic.

In [95]:
maximum(abs.(collect(output_many_models_gpu).-output_many_models_cpu))

2.8976820942716586e-14

2j. What do you predict for the speed-up factor of the GPU relative to the CPU?

In [96]:
response_2j = missing # md"Insert response"

missing

In [97]:
@benchmark wait(cpu_kernel_calc_rv_kepler_many_models!($output_many_models_cpu, $obs_data_small.t, $θ_eval.P, $θ_eval.K, $θ_eval.e, $θ_eval.ω, $θ_eval.M0, ndrange=( n_obs_small, num_models ) )) seconds=1

BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  33.424 ms …  34.097 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     33.533 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   33.587 ms ± 144.075 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

        █    ▁                                                  
  ▄▁▄▇▁▄█▇▄▇▁█▄▁▄▁▁▄▁▁▁▁▄▄▄▁▁▁▁▄▁▄▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  33.4 ms         Histogram: frequency by time         34.1 ms <

 Memory estimate: 4.42 KiB, allocs estimate: 62.

In [98]:
@benchmark wait(gpu_kernel_calc_rv_kepler_many_models!($output_many_models_gpu, $obs_data_small_gpu.t, $θ_eval_gpu.P, $θ_eval_gpu.K, $θ_eval_gpu.e, $θ_eval_gpu.ω, $θ_eval_gpu.M0, ndrange=( n_obs_small, num_models ) )) seconds=1

BenchmarkTools.Trial: 4335 samples with 1 evaluation.
 Range (min … max):  221.824 μs … 340.402 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     225.864 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   227.354 μs ±   4.982 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▃▆█▇▆▅▃▁                                                  
  ▂▃▄▅██████████▆▆▅▅▅▅▆▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▂▂▂▂▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂ ▃
  222 μs           Histogram: frequency by time          246 μs <

 Memory estimate: 3.77 KiB, allocs estimate: 66.

2k.  How did the benchmarking results compare to your prediction?  What might explain any differences?

In [99]:
response_2k = missing # md"Insert response"

missing

## Evaluating many models with reduction
We can likely get a further speed-up be performing the reduction operation on the GPU and returning only the goodness of fit statistics, rather than every predicted velocity.  We'll try that below.

In [100]:
function cpu_calc_chisq_kepler_many_models(t::AbstractArray, rv_obs::AbstractArray, σ::AbstractArray, θ; workspace = missing )
    @assert length(t) == length(rv_obs) == length(σ)
    @assert ismissing(workspace) || size(workspace) == (length(t), length(first(θ)) )
    rv_pred = ismissing(workspace) ? zeros(eltype(rv_obs),length(t),length(first(θ))) : workspace
    wait(cpu_kernel_calc_rv_kepler_many_models!(rv_pred, t, θ.P, θ.K, θ.e, θ.ω, θ.M0, ndrange=(length(t), length(first(θ))) ))
    χ² = vec(sum(((rv_pred.-rv_obs)./σ).^2, dims=1))
end

function cpu_calc_chisq_kepler_many_models(data::NamedTuple, θ; workspace = missing)
    cpu_calc_chisq_kepler_many_models(data.t, data.rv, data.σ, θ, workspace = workspace)
end

cpu_calc_chisq_kepler_many_models (generic function with 2 methods)

In [101]:
function gpu_calc_chisq_kepler_many_models(t::CuArray, rv_obs::CuArray, σ::CuArray, θ; workspace::Union{Missing,CuArray} = CUDA.zeros(eltype(rv_obs),length(t),length(θ.P)) )
    @assert length(t) == length(rv_obs) == length(σ)
    @assert ismissing(workspace) || size(workspace) == (length(t), length(first(θ)) )
    rv_pred = ismissing(workspace) ? CUDA.zeros(eltype(rv_obs),length(t),length(first(θ)))  : workspace
    (gpu_kernel_calc_rv_kepler_many_models!(rv_pred, t, θ.P, θ.K, θ.e, θ.ω, θ.M0, ndrange=(length(t), length(first(θ))) ))
    χ² = vec(sum(((rv_pred.-rv_obs)./σ).^2, dims=1))
end

function gpu_calc_chisq_kepler_many_models(t::AbstractArray, rv_obs::AbstractArray, σ::AbstractArray, θ; workspace = missing )
    t_d = isa(t,CuArray) ? t : convert(CuArray{Float64,1},t)
    rv_obs_d = isa(rv_obs,CuArray) ? rv_obs : convert(CuArray{Float64,1},rv_obs)
    σ_d = isa(σ,CuArray) ? σ : convert(CuArray{Float64,1},σ)
    workspace_d = isa(workspace, CuArray) ? workspace : CUDA.zeros(eltype(rv_obs),length(t_d),length(first(θ)))
    gpu_calc_chisq_kepler_many_models(t_d,rv_obs_d,σ_d,θ, workspace=workspace_d)
end

function gpu_calc_chisq_kepler_many_models(data::NamedTuple, θ; workspace = missing)
    gpu_calc_chisq_kepler_many_models(data.t, data.rv, data.σ, θ, workspace = workspace)
end

gpu_calc_chisq_kepler_many_models (generic function with 3 methods)

In [102]:
χ²s_gpu = gpu_calc_chisq_kepler_many_models(obs_data_small_gpu, θ_eval_gpu, workspace = output_many_models_gpu);
χ²s_cpu = cpu_calc_chisq_kepler_many_models(obs_data_small, θ_eval, workspace = output_many_models_cpu);
maximum(abs.(collect(χ²s_gpu).-χ²s_cpu))

5.4569682106375694e-12

In [103]:
@benchmark gpu_calc_chisq_kepler_many_models($obs_data_small_gpu, $θ_eval_gpu, workspace = $output_many_models_gpu) seconds=1

BenchmarkTools.Trial: 3330 samples with 1 evaluation.
 Range (min … max):   60.082 μs … 166.177 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      92.659 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   315.090 μs ±   4.550 ms  ┊ GC (mean ± σ):  0.87% ± 0.10%

  ▁█▁                                                            
  ███▃▂▂▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▄▅▆▆▄▃▄▅▅▃▂▂▂▂▁▁▁▁▁▁▂▂▁▂▁▁▁▁▁▁▂▂▂▂ ▃
  60.1 μs          Histogram: frequency by time          508 μs <

 Memory estimate: 11.78 KiB, allocs estimate: 192.

In [104]:
@benchmark χ²s_cpu = cpu_calc_chisq_kepler_many_models($obs_data_small, $θ_eval, workspace = $output_many_models_cpu) seconds=1

BenchmarkTools.Trial: 28 samples with 1 evaluation.
 Range (min … max):  34.421 ms … 46.661 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     35.265 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   35.839 ms ±  2.458 ms  ┊ GC (mean ± σ):  0.64% ± 2.16%

  █     ▄                                                      
  █▄▁▁▁▁█▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  34.4 ms         Histogram: frequency by time        46.7 ms <

 Memory estimate: 4.02 MiB, allocs estimate: 66.

2l. What do you observe for the speed-up factor of the GPU relative to the CPU now that we're performing the reduction on the GPU with a pre-allocated workspace?

In [105]:
response_2l = missing # md"Insert response"

missing

2m.  If you still have some time, try changing `num_models` and rerunning the affected cells.   How many models do you need to evaluate in parallel to get at least a factor of 50 performance improvement over the CPU?

In [106]:
response_2m = missing # md"Insert response"

missing

### Comparison to simple CPU version
Just for fun, here's a benchmark for the same computation on the CPU without kernel abstraction.

In [107]:
function get_nth_as_namedtuple(θ::NamedTuple{NTK,NTVT}, n::Integer) where { NTK, T<:Real, N1, A<:AbstractArray{T,N1}, N2, NTVT<:NTuple{N2,A} }
    @assert 1 <= n <= length(first(values(θ)))
    (; zip(keys(θ), map(x->x[n],values(θ)) )... )
end

get_nth_as_namedtuple (generic function with 1 method)

In [108]:
@elapsed map(n->calc_chisq_rv_kepler_cpu_simple(obs_data_small,get_nth_as_namedtuple(θ_eval,n)), 1:length(first(θ_eval)) )

0.27750318